<h1 style="text-align:center;background-color:DarkTurquoise; color:Teal; font-size: 40px">5. Sailing Towards Sentience
<img src="https://uploads-ssl.webflow.com/615bf2d50c4d2f10541ef9a1/61d48ffd38497b2c56571a0d_Univ.AI%20Logo_darkbg.svg" style="width:200px;height:200px;"></h1>
<div class="alert alert-info" role="alert">
  <b>Team :</b><br>
  <span style="color: Teal; font-weight: bold; font-size: 40px">The Sentient Chef </span><br>
  <span style="color: LightSeaGreen; font-weight: bold; font-size: 20px;">Univ.AI DS-1 C5 Final Project</span><br><br>
  <b>Voyage Controller :</b><br>
  <span style="color: LightSeaGreen; font-weight: bold;">Shibani Budhraja </span><br><br>
  <b>On Board Crews :</b><br>
  <span style="color: LightSeaGreen; font-weight: bold;">1. Swarnava Bhattacharjee</span><br>
  <span style="color: LightSeaGreen; font-weight: bold;">2. Karthik Rathod</span><br>
  <span style="color: LightSeaGreen; font-weight: bold;">3. Yamuna Katta</span><br>
  <span style="color: LightSeaGreen; font-weight: bold;">4. Venkatesh BY</span><br></div>

![recommendation](https://i.imgur.com/aUFb99f.jpeg)
The above image is built with AI model, ©️The Sentient Chef 

<div class="alert alert-warning" role="alert">
    <b>Decisions:</b> We considered the <b>gensim</b> library as it is open-source library for natural language processing (NLP) and topic modeling, which can also be used for recommendation systems.
    <br><br>
    <ul>
        <li><b>Efficient algorithms:</b> Gensim uses efficient algorithms for text processing, such as the Latent Dirichlet Allocation (LDA) algorithm, which can be used for topic modeling and can also be adapted for recommendation systems.</li>
        <li><b>Scalability:</b> Gensim is designed to handle large datasets and can scale well to large amounts of data, making it suitable for building recommendation systems that need to handle large volumes of user interactions and item data.</li>
    </ul>
</div> 

<h1 style="text-align:center;background-color:#117864; color:white" >Installing the gensim library for building Recommendation System.</h1>
<div class="alert alert-success" role="alert">
    <b> Gensim </b> is a free open-source Python library for representing documents as semantic vectors, as efficiently (computer-wise) and painlessly (human-wise) as possible. Gensim is designed to process raw, unstructured digital texts (“plain text”) using unsupervised machine learning algorithms.
</div>

In [ ]:
!pip install gensim

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import re
import collections
import warnings
import nltk
warnings.filterwarnings('ignore')

from gensim.corpora import Dictionary
from gensim.models.lsimodel import LsiModel
from gensim.similarities import MatrixSimilarity


<h2 style="text-align:center;background-color:#117864; color:white" >Defining Functions</h2>

In [ ]:
def convert_tolist(x):
    """
    Converts a string representation of a list into a list object.

    Args:
    - x (str): a string representation of a list, e.g., "['a', 'b', 'c']"

    Returns:
    - y (list): a list object created from the input string
    """
    
    y = eval(x)
    return y

In [ ]:
def remove_ingerdients(x):
    """
    Returns a filtered list of ingredients that appear more than 2 times in the input list.

    Args:
    - x (list): a list of strings representing ingredients

    Returns:
    - filtered_list (list): a list of strings representing ingredients that appear more than 2 times in the input list
    """
    filtered_list = []
    for i in x:
        try:
            if counts.get(i) > 2:
                filtered_list.append(i)
        except:
            pass
    return filtered_list

In [ ]:
def list_to_string(ingredient_list):
    """
    Converts a list of ingredients into a string with each ingredient separated by a space.

    Args:
        ingredient_list (list): A list of strings representing the ingredients.

    Returns:
        str: A string representation of the ingredients, with each ingredient separated by a space.

    Example:
        >>> ingredients = ["flour", "sugar", "butter", "eggs"]
        >>> list_to_string(ingredients)
        'flour sugar butter eggs'
    """
    return ' '.join(ingredient_list)

<h2 style="text-align:center;background-color:#117864; color:white" >Feature Preprocessing</h2>

In [ ]:
# Importing the requried csv file for recommendation
df = pd.read_csv("/kaggle/input/the-sentient-chef-notebook-eda/df_diet_type.csv")
df_diet = df[['title','ingredients','diet_type','description','URL']]
df_diet.shape

In [ ]:
# converting the datatype of the ingerdients data-points from string to list
df_diet.ingredients = df_diet.ingredients.apply(convert_tolist)

In [ ]:
# creating the list of all the ingerdients in the data frame to be used in filteration
list_ingeredients = []
for i in df_diet['ingredients']:
    for j in range(len(i)):
        list_ingeredients.append(i[j])
len(list_ingeredients)

In [ ]:
# Using nltk to seperate the words in the list by taging them based on parts of speech
pos_tags = nltk.pos_tag(list_ingeredients)

# selecting only those words which are tagged with noun
nouns = [word for (word, pos) in pos_tags if pos.startswith('N')]


In [ ]:
# there might be words with are not nouns and yet not discarded 
# to filter them out we will be dropping words whos length is less than 2 letters
filtered_words = [word for word in nouns if len(word) > 2]
len(filtered_words)

In [ ]:
# using counter from collections library we will get the frequeincy of the word in whole list
counts = collections.Counter(filtered_words)
len(counts.keys())

In [ ]:
# the counts dictionary created above will be used to filter the ingerdients from the individual recipes using the remove_ingerdients function
df_diet.ingredients = df_diet.ingredients.apply(remove_ingerdients)

In [ ]:
# saving the dataframe to be further use in model deployment
df_diet.to_csv("Recommendation_data.csv", index = False)

In [ ]:
# we will be dividing the dataset in three dataframe where division will be based on diet type of recipe
df_diet_non_veg = df_diet[df_diet['diet_type'] == 'nveg'].reset_index(drop = True).copy()
df_diet_veg = df_diet[df_diet['diet_type'] == 'veg'].reset_index(drop = True).copy()
df_diet_vegan = df_diet[df_diet['diet_type'] == 'vegan'].reset_index(drop = True).copy()


<h1 style="text-align:center;background-color:#117864; color:white" >Model Building :</h1>
<div class="alert alert-success" role="alert">
    The models below uses the Gensim library to create an <b>LSI (Latent Semantic Indexing)</b> topic model and a similarity index for a collection of recipe ingredients. The function takes a pandas DataFrame containing recipe ingredients, creates a bag-of-words representation of the ingredients, creates an LSI topic model from the bag-of-words representation, and creates a similarity index for the topic model. The similarity index will used to identify recipes that are most similar to a user's input ingredients.
</div>

In [ ]:
#Generating a dictionary from the one-hot encoded ingredients
ingredient_dict_0 = Dictionary(df_diet['ingredients'].apply(lambda x: x.split()))

#Generating a bag-of-words representation of the ingredient data
ingredient_bow = [ingredient_dict_0.doc2bow(recipe.split()) for recipe in df_diet['ingredients']]

#Creating an LSI topic model from the ingredient data
model_0 = LsiModel(corpus=ingredient_bow, id2word=ingredient_dict_0, num_topics=len(df_diet['ingredients']))

#Creating a matrix similarity index for the topic model
index_0 = MatrixSimilarity(model_0[ingredient_bow])

<h2 style="text-align:center;background-color:#1F618D; color:white" >SUPRISE-ME</h2>
<div class="alert alert-info" role="alert">
    This model will be recommendating from the data set conating all the recipes.
</div>

In [ ]:

#Generating a dictionary from the one-hot encoded ingredients
ingredient_dict_0 = Dictionary(df_diet['ingredients'].apply(lambda x: x.split()))

#Generating a bag-of-words representation of the ingredient data
ingredient_bow = [ingredient_dict_0.doc2bow(recipe.split()) for recipe in df_diet['ingredients']]

#Creating an LSI topic model from the ingredient data
model_0 = LsiModel(corpus=ingredient_bow, id2word=ingredient_dict_0, num_topics=len(df_diet['ingredients']))

#Creating a matrix similarity index for the topic model
index_0 = MatrixSimilarity(model_0[ingredient_bow])



<h2 style="text-align:center;background-color:#1F618D; color:white" >NON-VEG</h2>
<div class="alert alert-info" role="alert">
    This model will be recommendating from the data set conating all the Non_veg recipes
</div>

In [ ]:

#Generating a dictionary from the one-hot encoded ingredients
ingredient_dict_1 = Dictionary(df_diet_non_veg['ingredients'].apply(lambda x: x.split()))

#Generating a bag-of-words representation of the ingredient data
ingredient_bow = [ingredient_dict_1.doc2bow(recipe.split()) for recipe in df_diet_non_veg['ingredients']]

#Creating an LSI topic model from the ingredient data
model_1 = LsiModel(corpus=ingredient_bow, id2word=ingredient_dict_1, num_topics=len(df_diet_non_veg['ingredients']))

#Creating a matrix similarity index for the topic model
index_1 = MatrixSimilarity(model_1[ingredient_bow])



<h2 style="text-align:center;background-color:#1F618D; color:white" >VEG</h2>
<div class="alert alert-info" role="alert">
    This model will be recommendating from the data set conating all the veg recipes
</div>

In [ ]:

#Generating a dictionary from the one-hot encoded ingredients
ingredient_dict_2 = Dictionary(df_diet_veg['ingredients'].apply(lambda x: x.split()))

#Generating a bag-of-words representation of the ingredient data
ingredient_bow = [ingredient_dict_2.doc2bow(recipe.split()) for recipe in df_diet_veg['ingredients']]

#Creating an LSI topic model from the ingredient data
model_2 = LsiModel(corpus=ingredient_bow, id2word=ingredient_dict_2, num_topics=len(df_diet_veg['ingredients']))

#Creating a matrix similarity index for the topic model
index_2 = MatrixSimilarity(model_2[ingredient_bow])



<h2 style="text-align:center;background-color:#1F618D; color:white" >VEGAN</h2>
<div class="alert alert-info" role="alert">
    This model will be recommendating from the data set conating all the vegan recipes
</div>

In [ ]:

#Generating a dictionary from the one-hot encoded ingredients
ingredient_dict_3 = Dictionary(df_diet_vegan['ingredients'].apply(lambda x: x.split()))

#Generating a bag-of-words representation of the ingredient data
ingredient_bow = [ingredient_dict_3.doc2bow(recipe.split()) for recipe in df_diet_vegan['ingredients']]

#Creating an LSI topic model from the ingredient data
model_3 = LsiModel(corpus=ingredient_bow, id2word=ingredient_dict_3, num_topics=len(df_diet_vegan['ingredients']))

#Creating a matrix similarity index for the topic model
index_3 = MatrixSimilarity(model_3[ingredient_bow])


<h2 style="text-align:center;background-color:#5499C7; color:white" >This dictionary will be used to select the appropriate model for recommendation system</h2>

In [ ]:
model_dict = {"suprise":0,"non-veg":1,"veg":2,"vegan":3}

In [ ]:
# Define a function to get the top n similar recipes based on user ingredients
def get_recipes_by_ingredients(ingredients, n,model):
    """
    Given a list of ingredients, return a dataframe of n recommended recipes based on the similarity 
    of the ingredients to those in the dataset, for a specific diet type.

    Args:
    - ingredients (list): A list of ingredients.
    - n (int): The number of recommended recipes to return.
    - model (str): A string indicating the diet type to consider. 
                   Possible values: 'suprise', 'non-veg', 'veg', 'vegan'.

    Returns:
    - recommends (DataFrame): A Pandas DataFrame containing the n recommended recipes.
    """
    if model_dict.get(model) == 0:
        # Convert the user ingredients to bag-of-words representation
        ingredient_bow = ingredient_dict_0.doc2bow(ingredients)

        # Get the topic distribution for the user ingredients
        user_topics = model_0[ingredient_bow]

        # Get the similarity scores for all recipes based on user ingredients
        sim_scores = index_0[user_topics]

        # Sort the recipes by similarity score
        sim_scores = sorted(enumerate(sim_scores), key=lambda x: x[1], reverse=True)

        # Get the top n similar recipes
        sim_scores = sim_scores[0:n]
        recipe_indices = [i[0] for i in sim_scores]
        recommends = df_diet.iloc[recipe_indices]
        recommends = recommends.reset_index(drop = True)
        return recommends
    if model_dict.get(model) == 1:
        # Convert the user ingredients to bag-of-words representation
        ingredient_bow = ingredient_dict_1.doc2bow(ingredients)

        # Get the topic distribution for the user ingredients
        user_topics = model_1[ingredient_bow]

        # Get the similarity scores for all recipes based on user ingredients
        sim_scores = index_1[user_topics]

        # Sort the recipes by similarity score
        sim_scores = sorted(enumerate(sim_scores), key=lambda x: x[1], reverse=True)

        # Get the top n similar recipes
        sim_scores = sim_scores[0:n]
        recipe_indices = [i[0] for i in sim_scores]
        recommends = df_diet_non_veg.iloc[recipe_indices]
        recommends = recommends.reset_index(drop = True)
        return recommends
    if model_dict.get(model) == 2:
        # Convert the user ingredients to bag-of-words representation
        ingredient_bow = ingredient_dict_2.doc2bow(ingredients)

        # Get the topic distribution for the user ingredients
        user_topics = model_2[ingredient_bow]

        # Get the similarity scores for all recipes based on user ingredients
        sim_scores = index_2[user_topics]

        # Sort the recipes by similarity score
        sim_scores = sorted(enumerate(sim_scores), key=lambda x: x[1], reverse=True)

        # Get the top n similar recipes
        sim_scores = sim_scores[0:n]
        recipe_indices = [i[0] for i in sim_scores]
        recommends = df_diet_veg.iloc[recipe_indices]
        recommends = recommends.reset_index(drop = True)
        return recommends
    if model_dict.get(model) == 3:
        # Convert the user ingredients to bag-of-words representation
        ingredient_bow = ingredient_dict_3.doc2bow(ingredients)

        # Get the topic distribution for the user ingredients
        user_topics = model_3[ingredient_bow]

        # Get the similarity scores for all recipes based on user ingredients
        sim_scores = index_3[user_topics]

        # Sort the recipes by similarity score
        sim_scores = sorted(enumerate(sim_scores), key=lambda x: x[1], reverse=True)

        # Get the top n similar recipes
        sim_scores = sim_scores[0:n]
        recipe_indices = [i[0] for i in sim_scores]
        recommends = df_diet_vegan.iloc[recipe_indices]
        recommends = recommends.reset_index(drop = True)
        return recommends

In [ ]:

ingredients = ['chicken', 'cheese','butter','garlic']
get_recipes_by_ingredients(ingredients, n=10,model ='non-veg')

In [ ]:
ingredients = ['salmon','butter']
get_recipes_by_ingredients(ingredients, n=10,model ='veg')

<div class="alert alert-info" role="alert">
    The below code saves the <b>LSI models</b>, similarity indices, and dictionaries for each diet type as separate files, so they can be loaded and reused later in web deployment code.
</div>

In [ ]:
model_0.save('model_0')
index_0.save('index_0')
ingredient_dict_0.save('ingredient_dict_0.dict')
model_1.save('model_1')
index_1.save('index_1')
ingredient_dict_1.save('ingredient_dict_1.dict')
model_2.save('model_2')
index_2.save('index_2')
ingredient_dict_2.save('ingredient_dict_2.dict')
model_3.save('model_3')
index_3.save('index_3')
ingredient_dict_0.save('ingredient_dict_3.dict')

<nav aria-label="The Sentient Chef">
  <ul class="pagination">
    <li class="page-item"><a class="page-link" href="https://www.kaggle.com/code/karthikrathod/the-sentient-chef-scrapping-notebook">1. Setting Sail on the Recipe Hunt </a></li>
    <li class="page-item"><a class="page-link" href="#">2. Navigating the Choppy Waters of Preprocessing</a></li>
    <li class="page-item"><a class="page-link" href="https://www.kaggle.com/karthikrathod/the-sentient-chef-eda">3. Surveying the Seas of Recipe Data</a></li>
    <li class="page-item"><a class="page-link" href="https://www.kaggle.com/swarnava007/the-sentient-chef-maps">4. Mapping the Culinary World </a></li>
    <li class="page-item"><a class="page-link" href="https://www.kaggle.com/code/karthikrathod/recipes-recommendation">5. Sailing Towards Sentience 📌</a></li>
  </ul>
</nav>